In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
import plotly.express as px
import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
dataset = pd.read_csv('heart.csv')
dataset.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
dataset=dataset.drop(['fbs','slope','restecg','thal'],axis=1)



In [4]:
dataset.tail()

,age,sex,cp,trestbps,chol,thalach,exang,oldpeak,ca,target
298,57,0,0,140,241,123,1,0.2,0,0
299,45,1,3,110,264,132,0,1.2,0,0
300,68,1,0,144,193,141,0,3.4,2,0
301,57,1,0,130,131,115,1,1.2,1,0
302,57,0,1,130,236,174,0,0.0,1,0


In [5]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values
#X = X.astype(np.float64)
#y = y.astype(np.float64)

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
age         303 non-null int64
sex         303 non-null int64
cp          303 non-null int64
trestbps    303 non-null int64
chol        303 non-null int64
thalach     303 non-null int64
exang       303 non-null int64
oldpeak     303 non-null float64
ca          303 non-null int64
target      303 non-null int64
dtypes: float64(1), int64(9)
memory usage: 23.8 KB


In [7]:
#67,1,3,123,256,124,1,0.2,1

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=101)

In [9]:
from sklearn.linear_model import LogisticRegression
log1= LogisticRegression(C=1.0,penalty='l2',solver='newton-cg')
log1.fit(X_train,y_train)
errors = np.sqrt(mean_squared_error(y_test,log1.predict(X_test)))

In [10]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=log1, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 80.15 %
std is 8.52 %


In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
hyper = {'penalty':['l1', 'l2', 'elasticnet', 'none'],'C':[0.1,0.3,0.5,0.7,0.9,1.0,1.5],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

In [13]:
grid_search=GridSearchCV(estimator=log1,param_grid=hyper,cv=10,scoring='accuracy',n_jobs=-1)
grid_search.fit(X_train,y_train)
best_accuracy=grid_search.best_score_
print(best_accuracy)
best_param=grid_search.best_params_
print(best_param)

0.8305
{'C': 0.3, 'penalty': 'l2', 'solver': 'lbfgs'}


C:\Users\VIDYA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [14]:
from sklearn.metrics import confusion_matrix,accuracy_score
pre = log1.predict(X_test)

In [15]:
acc_log=accuracy_score(pre,y_test)

In [16]:
#trainpre = llm.predict(X_train)

In [17]:
#accuracy_score(trainpre,y_train)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
score=[]

for i in range(1,10):
    
    
    knn=KNeighborsClassifier(n_neighbors=i,metric='minkowski',p=2)
    knn.fit(X_train,y_train)
   

In [19]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=knn, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 64.82 %
std is 12.34 %


In [20]:
pre1 = knn.predict(X_test)

In [21]:
acc_knn=accuracy_score(pre1,y_test)

In [22]:
from sklearn.svm import SVC
svm_model=SVC(kernel='rbf',random_state=0)
svm_model.fit(X_train,y_train)

SVC(random_state=0)

In [23]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=svm_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 64.80 %
std is 7.26 %


In [24]:
pre2 = svm_model.predict(X_test)

In [25]:
acc_svmrbf=accuracy_score(pre2,y_test)

In [26]:
from sklearn.tree import DecisionTreeClassifier
decision_model=DecisionTreeClassifier(criterion='entropy',random_state=0)
decision_model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [27]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=decision_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 74.37 %
std is 5.55 %


In [28]:
pre3 = decision_model.predict(X_test)

In [29]:
acc_decision=accuracy_score(pre3,y_test)

In [30]:
from sklearn.ensemble import RandomForestClassifier
randomforest_model=RandomForestClassifier(n_estimators=55,criterion='entropy',random_state=0)
randomforest_model.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=55, random_state=0)

In [31]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=randomforest_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 81.43 %
std is 5.54 %


In [32]:
pre4 = randomforest_model.predict(X_test)

In [33]:
acc_random=accuracy_score(pre4,y_test)

In [34]:
from sklearn.svm import SVC
svmlinear_model=SVC(kernel='linear',random_state=0)
svmlinear_model.fit(X_train,y_train)

SVC(kernel='linear', random_state=0)

In [35]:

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=svmlinear_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 80.17 %
std is 7.16 %


In [36]:
pre5 = svmlinear_model.predict(X_test)

In [37]:
acc_svmlinear=accuracy_score(pre5,y_test)

In [38]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=xgb_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 77.68 %
std is 8.37 %


In [40]:
pre5 = xgb_model.predict(X_test)

In [41]:
acc_xgb=accuracy_score(pre5,y_test)

In [42]:
print(acc_log)
print(acc_knn)
print(acc_svmrbf)
print(acc_decision)
print(acc_random)
print(acc_svmlinear)
print(acc_xgb)

0.8688524590163934
0.6557377049180327
0.6065573770491803
0.8032786885245902
0.7868852459016393
0.8360655737704918
0.819672131147541


In [43]:
#logistic_model.predict([[68,1,0,144,193,141,0,3.4,2]])
#68	1	0	144	193	141	0	3.4	2

In [44]:
from sklearn.ensemble import AdaBoostClassifier
adaboost_model = AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=1))
adaboost_model.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=1))

In [45]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=adaboost_model, X=X_train ,y=y_train,cv=5)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 79.35 %
std is 2.48 %


In [46]:
from sklearn.ensemble import ExtraTreesClassifier
extra_model = ExtraTreesClassifier()
extra_model.fit(X_train,y_train)

ExtraTreesClassifier()

In [47]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=extra_model, X=X_train ,y=y_train,cv=5)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 80.98 %
std is 2.52 %


In [48]:
from sklearn.ensemble import GradientBoostingClassifier
gradiant_model = GradientBoostingClassifier()
gradiant_model.fit(X_train,y_train)

GradientBoostingClassifier()

In [49]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=gradiant_model, X=X_train ,y=y_train,cv=10)
print("accuracy is {:.2f} %".format(accuracies.mean()*100))
print("std is {:.2f} %".format(accuracies.std()*100))

accuracy is 78.12 %
std is 6.89 %


In [50]:
ada = adaboost_model.predict(X_test)
accuracy_score(y_test,ada)

0.8524590163934426

In [51]:
extra = extra_model.predict(X_test)
accuracy_score(y_test,extra)

0.8032786885245902

In [52]:
gra = gradiant_model.predict(X_test)
accuracy_score(y_test,gra)

0.8032786885245902

In [60]:
import pickle
pickle_out = open("logheart.pkl","wb")
pickle.dump(log1, pickle_out)
pickle_out.close()

In [61]:
dataset.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'thalach', 'exang', 'oldpeak',
       'ca', 'target'],
      dtype='object')

In [56]:
log1.predict([[56,1,1,120,236,178,0,0.8,0]])



array([1], dtype=int64)

In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pre))


              precision    recall  f1-score   support

           0       0.96      0.77      0.86        31
           1       0.81      0.97      0.88        30

    accuracy                           0.87        61
   macro avg       0.88      0.87      0.87        61
weighted avg       0.88      0.87      0.87        61



In [58]:
confusion_matrix(pre,y_test)

array([[24,  1],
       [ 7, 29]], dtype=int64)

In [59]:
from sklearn.metrics import roc_curve
roc_curve(y_test,pre)


(array([0.        , 0.22580645, 1.        ]),
 array([0.        , 0.96666667, 1.        ]),
 array([2, 1, 0], dtype=int64))

In [ ]:
from sklearn.metrics import roc_curve, auc

logistic_fpr, logistic_tpr, threshold = roc_curve(y_test, pre)
auc_logistic = auc(logistic_fpr, logistic_tpr)


plt.figure(figsize=(5, 5), dpi=100)
plt.plot(logistic_fpr, logistic_tpr, marker='.', label='Logistic (auc = %0.3f)' % auc_logistic)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')

plt.legend()

plt.show()

In [ ]:
print(classification_report(y_test,pre5))